In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#=======================  IMPORTING LIBRARIES   ==========================#
import pandas as pd
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.font_manager
%matplotlib inline
from collections import Counter
from geopy.geocoders import Nominatim
from folium import plugins
import plotly.express as px


#==============================   functions   ================================#

#importing coordinates with accident address

def listaloc(X): 
    loc = X.copy()
    lista = []
    for i in loc:
        location = geolocator.geocode(i)
        if location == None:
            pass
        else:
            # australia
            if -60 <= location.latitude <= 35 and 70 <= location.longitude <= 200:
                coor = [location.latitude, location.longitude]
                lista.append(coor)
            else:
                pass
    return lista


In [ ]:
#importando dados
df = pd.read_csv("../input/shark-attack-dataset/attacks.csv")
type(df)
#conection mapa
geolocator = Nominatim(user_agent="Shark")

In [ ]:
df.columns

In [ ]:
#===================================   rename columns    ========================================#
df.rename(columns ={'Sex ':'Sex'},inplace=True)
df.rename(columns ={'Species ':'Species'},inplace=True)
df.rename(columns ={'Fatal (Y/N)':'Fatal'},inplace=True)

In [ ]:
country = df['Country']
Counter(country).most_common(5)

# Countries with the highest number of cases

In [ ]:
country = country.dropna()
top_country = dict(Counter(country).most_common(10))
y_count= (top_country.keys())
x_count= top_country.values()
y_pos_count = np.arange(len(y_count))
#=========================================================#

plt.style.use('bmh')
plt.rcParams['figure.figsize'] = [12, 8]
fig, ax = plt.subplots()
ax.barh(y_pos_count, x_count, align='center')
ax.set_yticks(y_pos_count)
ax.set_yticklabels(y_count)
ax.invert_yaxis()  
ax.set_xlabel('Number of cases')
ax.set(xlim=[0, 2500])
ax.set_title('Countries with the highest number of cases')
plt.show()

In [ ]:
#analyzing data from australia
df = df[df.Country == "AUSTRALIA"]

In [ ]:
#Missing values
df.isnull().sum()

In [ ]:
print("Fatal =" ,dict(df['Fatal'].value_counts()))
print("Sex = ",dict(df['Sex'].value_counts()))
print("Area = ",dict(df['Area'].value_counts()))
print("Type = ",dict(df['Type'].value_counts()))

In [ ]:
df['Year'].describe()

In [ ]:
#========================  removing missing values    ========================#
df.drop(['Unnamed: 22',
         'Unnamed: 23',
         'Case Number',
         'Name',
         'Investigator or Source',
         'pdf',
         'href formula',
         'href',
         'Case Number.1',
         'Case Number.2',
         'Injury',
         'original order',
         'Time',
         'Date'], axis=1 , inplace=True)
df.drop(df.index[6401:] , inplace=True)
#=========================================   Location    ===========================================#
df_loc = df["Location"].dropna()
df_loc = df_loc.map('{},Australia'.format)
df_list_loc = sorted(set(list(df_loc)))
#===================================    completing data    ======================================#
df["Sex"].fillna('UNKNOWN', inplace = True)
df['Fatal'].fillna('UNKNOWN', inplace = True)
df['Activity'].fillna('UNKNOWN', inplace = True)
df['Location'].fillna('UNKNOWN', inplace = True)
df['Area'].fillna('UNKNOWN', inplace = True)
df['Age'].fillna('UNKNOWN', inplace = True)
df['Year'].fillna('UNKNOWN' , inplace = True)
df['Type'].fillna('UNKNOWN', inplace = True)
#===================================   adjusting values    ========================================#
df = df.replace({'Sex':{'M ':'M','N':'M'}})
df = df.replace({'Year':{5:1936,0:1903, 'UNKNOWN':1936}})
df = df.replace({'Type':{'Invalid':'UNKNOWN',
                         'Boat':'Watercraft',
                         'Questionable':'UNKNOWN',
                         'Unverified':'UNKNOWN'}})
df = df.replace({'Fatal':{' N':'N','N ':'N'}})
df = df.replace({'Area':{'Westerm Australia':'Western Australia',
                     'Northern Territory ':'Northern Territory',
                     'Victoria ':'Victoria',
                     'Torres Strait ':'Torres Strait',
                     'New South Wales ':'New South Wales',
                     'Queensland ':'Queensland'}})
#======================================   reset index    ===========================================#
df = df.reset_index()
df.drop(['index'], axis=1 , inplace=True)

In [ ]:
#missing values
df.isnull().sum()

In [ ]:
#lista = listaloc(df_list_loc) #import coordenadas

In [ ]:
mapa = folium.Map(location=[-25.27,133.77], #location australia
                    tiles='Stamen Terrain',
                    zoom_start=3.5,
                    min_zoom = 3.5)

In [ ]:
coor = pd.read_csv("../input/shark-attack-dataset/list_coor_australia.csv")

# Accident site

In [ ]:
mapa.add_child(plugins.HeatMap(coor))
mapa

In [ ]:
df

# Accidents by sex

In [ ]:
#============================   accidents / sex =================================#
sex = df['Sex']
sex = dict(sex.value_counts())
Sexo = list(sex.keys())
Quant = list(sex.values())
ax=sns.barplot(x=Sexo, y=Quant, palette="bright",).set(xlabel='Sex', ylabel='amount', 
                                                      title="Accidents by sex (AUSTRALIA)")

# Survived / Died

In [ ]:
#============================  Survived / Died ================================#
fatal = df['Fatal']
fatal = dict(fatal.value_counts())
Fatal = list(fatal.keys())
Fatal[0]= "Survived"
Fatal[1]= "Died"
Quant_fatal = list(fatal.values())
Porcentagem = []
explode = (0.04,0.04,0.04)
for i in Quant_fatal:
    Porcentagem.append(int(i * 100/df['Fatal'].count()))
    
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 15
fig1, ax1 = plt.subplots()
ax1.pie(Porcentagem, explode=explode, labels=Fatal, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.set_title('Survived / Died')
plt.axis('equal')
plt.legend(loc='upper right')

# Attacks per year

In [ ]:
year = dict(df['Year'].value_counts())
year = dict(sorted(year.items()))
year_cop=dict(year)
type(year_cop)
for x, y in year.items():
    if x<1950:
        del year_cop[x]
    else:
        pass
ax1=plt.plot(year_cop.keys(), year_cop.values(), 'b-')
plt.title('Attacks per year')

# Most dangerous sharks in australia

In [ ]:
sharkc = df.groupby(['Species'],as_index=False).size()
sharkc = sharkc.sort_values(by=['size'], ascending=False)
sharkc = sharkc[0:4]
mlabels=['White shark','Tiger shark','Bull shark','Wobbegong shark']
plt.pie(sharkc['size'],labels=mlabels,explode = (0.02,0.02,0.02,0.02),autopct='%1.1f%%')
plt.title('Percentage share between the 4 most dangerous sharks in australia')
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.legend(loc=2)

In [ ]:
area = df['Area']
area = dict(Counter(area).most_common(10))
del area['UNKNOWN']
NSW = [-31.840233, 145.612793]
QLD = [-23.0 ,143.0]
WA = [-25.042261, 117.793221]
SA = [-30.000233, 136.209152]
VIC = [-37.020100,144.964600]
TS = [-9.874829834 ,142.587664316]
TAS =[-42.0, 147.0]
NT = [ -19.491411, 132.550964]
TOC = [-12.1709,  96.8417]
Color= '#2F4F4F'
tooltip="Click me"
print(area)

# Cases by Australian States and Territories

In [ ]:
m = folium.Map(location=[-25.27,133.77], #location australia
                  tiles='openstreetmap',
                  zoom_start=3.5,
                  min_zoom = 3.5,)
folium.CircleMarker(
    location=NSW,
    radius=50,
    popup="New South Wales:<b>496<b>",
    tooltip=tooltip,
    color=Color,
    fill=True,
    fill_color=Color,
).add_to(m)
folium.CircleMarker(
    location=QLD,
    radius=32,
    popup="Queensland:<b>322<b>",
    tooltip=tooltip,
    color=Color,
    fill=True,
    fill_color=Color,
).add_to(m)
folium.CircleMarker(
    location=WA,
    radius=20,
    popup="Western Australia:<b>196<b>",
    tooltip=tooltip,
    color=Color,
    fill=True,
    fill_color=Color,
).add_to(m)
folium.CircleMarker(
    location=SA,
    radius=11,
    popup="South Australia:<b>104<b>",
    tooltip=tooltip,
    color=Color,
    fill=True,
    fill_color=Color,
).add_to(m)
folium.CircleMarker(
    location=VIC,
    radius=12,
    popup="Victoria:<b>92<b>",
    tooltip=tooltip,
    color=Color,
    fill=True,
    fill_color=Color,
).add_to(m)
folium.CircleMarker(
    location=TS,
    radius=7,
    popup="Torres Strait:<b>72<b>",
    tooltip=tooltip,
    color=Color,
    fill=True,
    fill_color=Color,
).add_to(m)
folium.CircleMarker(
    location=TAS,
    radius=8,
    popup="Tasmania:<b>42<b>",
    tooltip=tooltip,
    color=Color,
    fill=True,
    fill_color=Color,
).add_to(m)
folium.CircleMarker(
    location=NT ,
    radius=6,
    popup="Northern Territory':<b>25<b>",
    tooltip=tooltip,
    color=Color,
    fill=True,
    fill_color=Color,
).add_to(m)
folium.CircleMarker(
    location=TOC,
    radius=3,
    popup="Territory of Cocos Islands:<b>1<b>",
    tooltip=tooltip,
    color=Color,
    fill=True,
    fill_color=Color,
).add_to(m)
m